In [1]:
import sqlite3
import random
from datetime import datetime, timedelta

# Connect to SQLite database (or create it if it doesn't exist)
conn = sqlite3.connect("sales.db")
cursor = conn.cursor()

# Create table for product information
cursor.execute(
    """
CREATE TABLE IF NOT EXISTS Product (
    product_id INTEGER PRIMARY KEY,
    product_name TEXT NOT NULL,
    category TEXT,
    price REAL
);
"""
)

# Create table for sales information
cursor.execute(
    """
CREATE TABLE IF NOT EXISTS Sales (
    sale_id INTEGER PRIMARY KEY,
    product_id INTEGER,
    sale_date DATE,
    quantity_sold INTEGER,
    FOREIGN KEY (product_id) REFERENCES Product(product_id)
);
"""
)

# Create table for product inventory
cursor.execute(
    """
CREATE TABLE IF NOT EXISTS Inventory (
    product_id INTEGER,
    inventory_date DATE,
    opening_stock INTEGER,
    closing_stock INTEGER,
    FOREIGN KEY (product_id) REFERENCES Product(product_id),
    PRIMARY KEY (product_id, inventory_date)
);
"""
)

# Insert dummy data into Product table
products = [
    (
        i,
        f"{chr(65 + (i % 26))}",
        f"Category {i % 5 + 1}",
        round(random.uniform(5, 50), 2),
    )
    for i in range(1, 11)
]
cursor.executemany(
    """
INSERT INTO Product (product_id, product_name, category, price)
VALUES (?, ?, ?, ?)
""",
    products,
)

# Generate dummy data for 3 months Sales table
sales = []
date_start = datetime.strptime("2025-01-01", "%Y-%m-%d")
sale_id = 1

for i in range(1, 11):
    for j in range(90):  # 90 days of sales data for each product
        sale_date = date_start + timedelta(days=j)
        quantity_sold = random.randint(1, 20)
        sales.append((sale_id, i, sale_date.strftime("%Y-%m-%d"), quantity_sold))
        sale_id += 1

cursor.executemany(
    """
INSERT INTO Sales (sale_id, product_id, sale_date, quantity_sold)
VALUES (?, ?, ?, ?)
""",
    sales,
)

# Generate dummy data for 3 months Inventory table
inventory = []

for i in range(1, 11):
    opening_stock = random.randint(100, 500)
    for month in range(3):  # 3 months of inventory data
        month_start = date_start + timedelta(days=month * 30)
        for j in range(30):  # 30 days per month
            inventory_date = month_start + timedelta(days=j)
            closing_stock = max(0, opening_stock - random.randint(1, 20))
            inventory.append(
                (i, inventory_date.strftime("%Y-%m-%d"), opening_stock, closing_stock)
            )
            opening_stock = closing_stock

        # Restock at the end of the month
        if month < 2:  # No restock after the last month
            opening_stock += random.randint(100, 500)

cursor.executemany(
    """
INSERT INTO Inventory (product_id, inventory_date, opening_stock, closing_stock)
VALUES (?, ?, ?, ?)
""",
    inventory,
)

# Commit changes
conn.commit()

# Close the connection
conn.close()

In [4]:
import pandas as pd

conn = sqlite3.connect("sales.db")
cursor = conn.cursor()

print("\nInventory Status for Product 1:")
cursor.execute("SELECT * FROM Sales WHERE product_id = 1")
data = cursor.fetchall()
df = pd.DataFrame(data, columns=["sale_id", "product_id", "sale_date", "quantity_sold"])


# Close the connection
conn.close()


Inventory Status for Product 1:


In [5]:
df.head(34)

,sale_id,product_id,sale_date,quantity_sold
0,1,1,2025-01-01,9
1,2,1,2025-01-02,15
2,3,1,2025-01-03,8
3,4,1,2025-01-04,14
4,5,1,2025-01-05,18
5,6,1,2025-01-06,2
6,7,1,2025-01-07,10
7,8,1,2025-01-08,20
8,9,1,2025-01-09,15
9,10,1,2025-01-10,12


In [7]:
import pandas as pd

conn = sqlite3.connect("sales.db")
cursor = conn.cursor()

cursor.execute("SELECT * FROM Product")
data = cursor.fetchall()
df = pd.DataFrame(data)


# Close the connection
conn.close()

In [8]:
df

,0,1,2,3
0,1,B,Category 2,19.05
1,2,C,Category 3,49.78
2,3,D,Category 4,22.12
3,4,E,Category 5,22.93
4,5,F,Category 1,14.90
5,6,G,Category 2,15.34
6,7,H,Category 3,23.68
7,8,I,Category 4,18.99
8,9,J,Category 5,27.48
9,10,K,Category 1,44.08
